
# ADS‑B @ 1090 MHz — RTL‑SDR Capture → Detect → Decode (Student Starter)

**Goal:** Minimal, well‑commented notebook for students to process RTL‑SDR captures of ADS‑B (Mode‑S) signals.  
**Pipeline:** IQ (CU8) → DC removal → Magnitude → (optional) oversample → Preamble detection → PPM bit slicing → CRC/PI check → Frame summary.

> This notebook uses `adsb_lib_v3.py` (standalone) located next to this notebook.



## 0. Requirements & Capture Instructions

**Hardware:** RTL‑SDR (Blog v3/v4 or equivalent) + simple 1090 MHz antenna (ideally external or near a window).  
**Software:** `rtl_sdr` (from rtl-sdr/airspy packages), Python 3, NumPy, Matplotlib.

**Example capture (2.0 MS/s, 3 seconds, manual gain):**
```bash
rtl_sdr -f 1090000000 -s 2000000 -g 49.6 -n $((3*2000000*2)) capture_1090.cu8
```
- `-f 1090000000` : center frequency 1090 MHz  
- `-s 2000000` : sample rate 2.0 MS/s (try 2.4 MS/s if stable)  
- `-g 49.6` : tuner gain in dB (try a few values or `-g 0` for auto if supported)  
- `-n` : number of **bytes** to record (2 per complex sample → duration × fs × 2)  
- Output: **interleaved CU8** (I then Q), named `capture_1090.cu8`

> Tip: Start with ~3–5 s to keep files small, then scale up.



## 1. Imports & Library Check
This notebook prefers using the library over re‑implementing functions here.


In [ ]:

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import importlib.util, sys

# Load adsb_lib_v3.py from the same folder
# lib_path = Path('adsb_lib_v3.py')
# assert lib_path.exists(), "adsb_lib_v3.py not found next to this notebook."
# spec = importlib.util.spec_from_file_location('adsb_lib_v3', lib_path)
# adsb = importlib.util.module_from_spec(spec)
# sys.modules['adsb_lib_v3'] = adsb
# spec.loader.exec_module(adsb)

# Load the FAST library instead of adsb_lib_v3.py
lib_path = Path('adsb_lib_v3fast.py')  # use v3fast
spec = importlib.util.spec_from_file_location('adsb_lib_v3fast', lib_path)
adsb = importlib.util.module_from_spec(spec)
sys.modules['adsb_lib_v3fast'] = adsb
spec.loader.exec_module(adsb)

# Quick peek at available functions
[name for name in dir(adsb) if not name.startswith('_')]



## 2. Configure Your Capture
Set the path to your IQ file and core parameters.


In [ ]:

# ---- Student Configuration ----
iq_path = Path('capture_1090.cu8')   # change if your filename differs
fmt = 'cu8'                          # 'cu8' for rtl_sdr raw, or 'cf32' for float32
fs = 2_000_000                       # sample rate used during capture (Hz)
max_samples = 4_000_000              # limit for quick tests; set None to load all

# Detection/decoding options
up = 2            # oversampling factor (1 = off). Try 2..6 for robustness.
min_snr = 6.0     # preamble detection threshold heuristic
cfg = adsb.DecodeConfig(up=up, min_snr=min_snr)

print(f"Using fs={fs} Hz, up={up}, min_snr={min_snr}")



## 3. Load → Preprocess → Detect → Decode
This uses the convenience `quick_pipeline` to run the full chain.


In [ ]:

# Load IQ (CU8 or CF32) and run the quick pipeline
iq = adsb.load_iq(iq_path, fmt=fmt, fs=fs, max_samples=max_samples)
dets, frames, mag_up, fs_up = adsb.quick_pipeline(iq, up=up, min_snr=min_snr, cfg=cfg)
print(f"Detections: {len(dets)} | Frames decoded (candidates): {len(frames)}")



## 4. Frame Summary
Quick look at the first few decoded frames.


In [ ]:

def icao_hex(v):
    return f"{v:06X}" if v is not None else "--"

print("p0	len	DF	ICAO	CRC_OK	HEX[:28]...")
for fr in frames[:20]:
    crc_str = "?" if fr.crc_ok is None else ("OK" if fr.crc_ok else "BAD")
    print(f"{fr.p0}	{fr.length_bits}	{fr.df if fr.df is not None else '--'}	{icao_hex(fr.icao)}	{crc_str}	{fr.hex[:28]}...")



## 5. Visualize First Detection (Time Domain)
Plot magnitude around the first detected preamble for sanity‑check.


In [ ]:

if len(dets):
    p0 = dets[0].p0
    win = 800  # samples around preamble (at fs_up)
    s = max(0, p0 - win//2)
    e = min(len(mag_up), p0 + win//2)
    x = np.arange(s, e)
    plt.figure()
    plt.plot(x, mag_up[s:e])
    plt.title(f'Magnitude around preamble @ {p0} (fs_up={fs_up:.0f} Hz)')
    plt.xlabel('Sample index')
    plt.ylabel('Magnitude')
    plt.grid(True)
else:
    print("No detections to plot.")



## 6. Optional: Quick PSD (Frequency Domain)
A coarse power spectral density (PSD) to check DC offset or front‑end issues.


In [ ]:

# Compute a simple PSD over a short slice
N = min(1_048_576, len(iq.samples))  # up to ~1M samples
x = iq.samples[:N] - np.mean(iq.samples[:N])
X = np.fft.fftshift(np.fft.fft(x))
psd = 20*np.log10(1e-9 + np.abs(X)/np.sqrt(N))

freq = np.fft.fftshift(np.fft.fftfreq(N, d=1/iq.fs))
plt.figure()
plt.plot(freq/1e6, psd)
plt.xlabel('Frequency (MHz) relative to center')
plt.ylabel('PSD (dB, arbitrary)')
plt.title('Coarse PSD of capture (centered)')
plt.grid(True)



## 7. (Optional) Synthetic Sanity Test
Build a synthetic ADS‑B frame waveform, add noise, and run detection/decoding.


In [ ]:

bits = adsb.synth_adsb_bits(112)  # random 112-bit payload (no real DF/ME semantics)
w = adsb.bits_to_ppm_wave(bits, fs=fs)          # baseband-like PPM envelope
y = adsb.pad_and_add_noise(w, snr_db=12.0)      # add noise + padding
mag_syn = np.abs(y).astype(np.float32)
mag_syn_up, fs_syn_up = adsb.mag_oversample(mag_syn, fs, up=2)

dets_syn = adsb.detect_preambles(mag_syn_up, fs_syn_up, min_snr=5.5, up=2)
frames_syn = adsb.decode_frames(mag_syn_up, fs_syn_up, dets_syn, cfg=adsb.DecodeConfig(up=2, min_snr=5.5))
print(f"SYNTH: detections={len(dets_syn)} frames={len(frames_syn)}")
if frames_syn:
    print('First hex:', adsb.bits_to_hex(frames_syn[0].bits)[:28]+'...')



---

## 8. Troubleshooting
- **No detections?** Try higher `-g` (gain) during capture, larger `up` (e.g., 4), or lower `min_snr` (e.g., 5.0).  
- **Few CRC‑OK frames?** Antenna placement matters. Try near a window or use a 1090 MHz filter/LNA.  
- **PSD shows a big spike at 0 Hz?** That’s DC offset; `dc_remove` mitigates it.

## 9. Mini‑Exercises
1. Compare results at `fs=2.0 MS/s` vs `2.4 MS/s` on the same location/time. Which gives more decodes? Why?  
2. Sweep `min_snr` from 4.5 to 7.5 and record detections/CRC‑OK counts. Plot the trend.  
3. Change `up` from 1→2→4. How does it impact detection robustness vs runtime?  
4. Parse DF=17 frames only and histogram the top 10 ICAO addresses received during a 2‑minute capture.

> Remember to **explain observations**: link them to SNR, bandwidth, multipath, and front‑end gain.
